# ParaView Hello Molecule Test
This notebook reads a timeseries of molecular trajectories.
Then, it creates a ParaView display widget showing the primary render view.
The notebook further demonstrates how we may use interaction widgets (sliders), to change the timestep.
Lastly, we switch to raytracing and path-tracing mode to demonstrate a photo-realistic rendering

Tested: Mon May  3 15:45:28 CEST 2021

In [ ]:
from paraview.simple import *

Version = str(GetParaViewVersion())

info = GetOpenGLInformation(location=servermanager.vtkSMSession.RENDER_SERVER)
print("ParaView Version ", Version)
print("Vendor:   %s" % info.GetVendor())
print("Version:  %s" % info.GetVersion())
print("Renderer: %s" % info.GetRenderer())

In [ ]:
molecule1 = XYZReader(FileName='/users/jfavre/Projects/Rizzi/release_H2_ex.xyz')
nb_of_timesteps = len(molecule1.TimestepValues)
print("Molecule trajectories with ", nb_of_timesteps, " steps")

In [ ]:
computeMoleculeBonds1 = ComputeMoleculeBonds(Input=molecule1)
computeMoleculeBonds1.UpdatePipeline()
computeMoleculeBonds1Display = Show(computeMoleculeBonds1, GetActiveView())

atomicNumbersLUT = GetColorTransferFunction('AtomicNumbers')
atomicNumbersLUT.InterpretValuesAsCategories = 1
atomicNumbersLUT.AnnotationsInitialized = 1
atomicNumbersLUT.ShowCategoricalColorsinDataRangeOnly = 1

atomicNumbersLUTColorBar = GetScalarBar(atomicNumbersLUT, GetActiveView())
atomicNumbersLUTColorBar.Title = ''
atomicNumbersLUTColorBar.ComponentTitle = ''

# set color bar visibility
atomicNumbersLUTColorBar.Visibility = 1
atomicNumbersLUTColorBar.Orientation = 'Horizontal'
atomicNumbersLUTColorBar.WindowLocation = 'AnyLocation'
atomicNumbersLUTColorBar.Position = [0.65, 0.075]
atomicNumbersLUTColorBar.ScalarBarLength = 0.3

In [ ]:
# create a new 'Annotate Time Filter'
annotateTimeFilter1 = AnnotateTimeFilter(Input=computeMoleculeBonds1)
annotateTimeFilter1Display = Show(annotateTimeFilter1, GetActiveView())

In [ ]:
from ipyparaview.widgets import PVDisplay
renderView1 = GetActiveView()
pvdisp = PVDisplay(renderView1)
w = display(pvdisp)

In [ ]:
GetActiveCamera().Azimuth(30)
GetActiveCamera().Elevation(15)

In [ ]:
# Interact from ipywidgets gives us a simple way to interactively control values
# with a callback function
from ipywidgets import interact, IntSlider

# set a new time-step
def time_slider(t):
    GetActiveView().ViewTime = t
    
i = interact(time_slider, t=IntSlider(min=0, max=nb_of_timesteps-1, step=1, value=0))

In [ ]:
def SaveImage(filename):
  from vtk import vtkPNGWriter
  img_writer = vtkPNGWriter()
  img_writer.SetInputConnection(pvdisp.w2i.GetOutputPort())
  img_writer.SetFileName(filename)
  img_writer.Write()

In [ ]:
# Add three planes around the molecule
bounds = computeMoleculeBonds1.GetDataInformation().GetBounds()
mat1 = "Metal_Scratched"
mat1 = "Metal_Stainless_Steel_mirror"
# add P percent of each dimensions. P > 0
P = 0.3
xoffset = (bounds[1]-bounds[0])*P
yoffset = (bounds[3]-bounds[2])*P
zoffset = (bounds[5]-bounds[4])*P

pxy = Plane(registrationName='Plane-XY')
pxy.Origin = [bounds[0]-xoffset, bounds[2]-yoffset, bounds[4]-zoffset]
pxy.Point1 = [bounds[1]+xoffset, bounds[2]-yoffset, bounds[4]-zoffset]
pxy.Point2 = [bounds[0]-xoffset, bounds[3]+yoffset, bounds[4]-zoffset]
r1=Show(pxy)
r1.OSPRayMaterial = mat1

pyz = Plane(registrationName='Plane-YZ')
pyz.Origin = pxy.Origin
pyz.Point1 = [bounds[0]-xoffset, bounds[3]+yoffset, bounds[4]-zoffset]
pyz.Point2 = [bounds[0]-xoffset, bounds[2]-yoffset, bounds[5]+zoffset]
r2=Show(pyz)
r2.OSPRayMaterial = 'None'

pxz = Plane(registrationName='Plane-XZ')
pxz.Origin = pxy.Origin
pxz.Point1 = [bounds[1]+xoffset, bounds[2]-yoffset, bounds[4]-zoffset]
pxz.Point2 = [bounds[0]-xoffset, bounds[2]-yoffset, bounds[5]+zoffset]
r3=Show(pxz)
r3.OSPRayMaterial = 'PBR_Brick_Wall_1'

In [ ]:
renderView1.EnableRayTracing = 1
renderView1.Denoise = 1
renderView1.BackEnd = 'OSPRay raycaster'
renderView1.ProgressivePasses=1000
renderView1.Shadows = 1

In [ ]:
if info.GetVendor().split()[0] == 'NVIDIA':
  renderView1.BackEnd = 'OSPRay pathtracer'
else:
  renderView1.BackEnd = 'OSPRay pathtracer'

In [ ]:
# Image generation below, using frequency=1, takes 2-3 minutes. You may speed that up by changing frequency to 10 for a quick test

In [ ]:
renderView1.ProgressivePasses=1
renderView1.SamplesPerPixel=10
# save all images to disk and encode into a movie
frequency=10
for i, ti in enumerate(range(0,len(molecule1.TimestepValues), frequency)):
    renderView1.ViewTime = ti
    SaveImage(format("/dev/shm/molecule_%06d.png" % i))

In [ ]:
# use a good quality value, here we use 9
!ffmpeg -i /dev/shm/molecule_%06d.png -c:v libtheora -q:v 9 /dev/shm/molecule.ogv

In [ ]:
from ipywidgets import Video

video = Video.from_file("/dev/shm/molecule.ogv")
video